In [2]:
from calendar import calendar
import pandas as pd
import json
import requests
import geopandas as gpd
import timeit
import re
# from sqlalchemy.orm import Session,sessionmaker
# Using SQLAlchemy to connect to the Database
from pathlib import Path
from sqlalchemy import create_engine

from sqlalchemy import create_engine,MetaData,event
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

from pandas.io.json import json_normalize

import secrets as Config

# from .utils.log_helper import *

from secrets import *
engine = create_engine(Config.API_DB_URI, echo=False)

Session = sessionmaker(autocommit=False, autoflush=False, bind=engine)

session = Session()
target_schema = "metro_api_dev"
Base = declarative_base(metadata=MetaData(schema=target_schema))

def get_db():
    db = Session()
    try:
        print('Connected to the database')
        yield db
    finally:
        db.close()

# routes_from_db = pd.read_sql_table('routes',get_db())
routes_from_db = pd.read_sql_query('select * from "routes"',con=engine)
routes_unique = routes_from_db
# new_df[~new_df['route_id']].str.contains(,na=False)


def get_route_code(row):
    if re.search("^[89].*\d\d$", row['route_id']):
        val = row['route_id'].split('-')[0]
    elif re.search("^[a-zA-Z]", row['route_id']):
        val = row['route_id']
    else:
        val = row['route_short_name']
    return val

def duplicate_route_code_rows_with_slashes(row):
    if re.search("/", row['route_code']):
        val = row['route_code'].split('/')
    return val

routes_unique['route_code'] = routes_unique.apply(lambda row: get_route_code(row), axis=1)

routes_sister_lines_df = routes_unique[routes_unique['route_code'].str.contains("/",na=False)]
routes_unique.drop(routes_unique[routes_unique['route_code'].str.contains("/",na=False)].index, inplace=True)
routes_sister_lines_df['route_code'] = routes_sister_lines_df.apply(lambda row: duplicate_route_code_rows_with_slashes(row), axis=1)
routes_sister_lines_df = routes_sister_lines_df.explode('route_code')
cleaned_routes_df = pd.concat([routes_unique,routes_sister_lines_df],ignore_index=True)
cleaned_routes_df


C:\Users\hikou\AppData\Local\Temp\ipykernel_19340\220812450.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  routes_sister_lines_df['route_code'] = routes_sister_lines_df.apply(lambda row: duplicate_route_code_rows_with_slashes(row), axis=1)


,route_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,route_url,agency_id,route_code
0,2-13167,2,Metro Local Line,WESTWOOD - EXPOSITION PARK VIA SUNSET-ALVARADO,3,None,None,None,LACMTA,2
1,4-13167,4,Metro Local Line,DOWNTOWN LA - SANTA MONICA VIA SANTA MONICA BL,3,None,None,None,LACMTA,4
2,16-13167,16,Metro Local Line,DOWNTOWN LA - WEST HOLLYWOOD VIA WEST 3RD ST,3,None,None,None,LACMTA,16
3,18-13167,18,Metro Local Line,WILSHIRE WESTERN STA-MONTEBELLO VIA 6TH - WHIT...,3,None,None,None,LACMTA,18
4,20-13167,20,Metro Local Line,DOWNTOWN LA - SANTA MONICA VIA WILSHIRE BL,3,None,None,None,LACMTA,20
...,...,...,...,...,...,...,...,...,...,...
122,211-13167,211/215,Metro Local Line,INGLEWOOD-SOUTH BAY GALLERIA VIA PRAIRIE-INGLE...,3,None,None,None,LACMTA,215
123,242-13167,242/243,Metro Local Line,NORTHRIDGE - TARZANA VIA TAMPA - WINNETKA AVS,3,None,None,None,LACMTA,242
124,242-13167,242/243,Metro Local Line,NORTHRIDGE - TARZANA VIA TAMPA - WINNETKA AVS,3,None,None,None,LACMTA,243
125,487-13167,487/489,Metro Express Line,DOWNTOWN LA - SIERRA MADRE VILLA STA,3,None,None,None,LACMTA,487


In [3]:

resp = requests.get('https://www.metro.net/wp-json/wp/v2/line-override/', timeout=10,headers={'Content-Type': 'application/json'})
df= pd.read_json(resp.text)

word_press_line_data = json_normalize(df['acf'])

def duplicate_alt_id_rows_with_slashes(row):
    if re.search("/", row['alt_id']):
        val = row['alt_id'].split('/')
    return val

word_press_line_data.to_csv('custom-schedule-page.csv',index=False)
word_press_line_data.rename(columns={'route_id':'alt_id','pdf_file.url':'pdf_file_url','pdf_file.link':'pdf_file_link'},inplace=True)
word_press_line_data.drop(['pdf_file.ID','pdf_file.id','pdf_file.title','pdf_file.filename','pdf_file.filesize','pdf_file.alt','pdf_file.author','pdf_file.description','pdf_file.caption','pdf_file.name','pdf_file.status','pdf_file.uploaded_to','pdf_file.date','pdf_file.modified','pdf_file.menu_order','pdf_file.mime_type','pdf_file.type','pdf_file.subtype','pdf_file.icon'],axis=1,inplace=True)

new_word_press_line_data = word_press_line_data

word_press_line_data_sister_lines = word_press_line_data[word_press_line_data['alt_id'].str.contains("/",na=False)]
word_press_line_data.drop(word_press_line_data[word_press_line_data['alt_id'].str.contains("/",na=False)].index, inplace=True)
word_press_line_data_sister_lines['line_id'] = word_press_line_data_sister_lines.apply(lambda row: duplicate_alt_id_rows_with_slashes(row), axis=1)
word_press_line_data_sister_lines = word_press_line_data_sister_lines.explode('line_id')

combined_wordpress_data = pd.concat([word_press_line_data,word_press_line_data_sister_lines],ignore_index=True)
combined_wordpress_data.to_csv('combined_wordpress_data.csv',index=False)
merged_df = cleaned_routes_df.merge(combined_wordpress_data, left_on='route_code', right_on='line_id', how='left')

target_schema = "metro_api_dev"
merged_df.to_sql('route_overview',con=engine,if_exists='replace',index=False,schema=target_schema)

target_schema = "metro_api"
merged_df.to_sql('route_overview',con=engine,if_exists='replace',index=False,schema=target_schema)



C:\Users\hikou\AppData\Local\Temp\ipykernel_19340\3122220146.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  word_press_line_data = json_normalize(df['acf'])
C:\Users\hikou\AppData\Local\Temp\ipykernel_19340\3122220146.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  word_press_line_data_sister_lines['line_id'] = word_press_line_data_sister_lines.apply(lambda row: duplicate_alt_id_rows_with_slashes(row), axis=1)
